In [1]:
import collections

stop_words = []
with open("./stop_words.txt", "r", encoding='utf-8') as f_stopwords:
    for line in f_stopwords:
        line = line.replace("\r", "").replace("\n", "").strip()
        stop_words.append(line)
stop_words = set(stop_words)
print(len(stop_words))

1909


In [2]:
import re
import jieba

chinese_regular = u"([\u4e00-\u9fa5]+)"
pattern = re.compile(chinese_regular)

f_writer = open("./data/seg_The_Smiling_Proud_Wanderer.txt", "w", encoding="utf-8")
raw_word_list = []
with open("./data/The_Smiling_Proud_Wanderer.txt", "r", encoding="utf-8") as f_reader:
    for line in f_reader:
        line = line.replace("\r", "").replace("\n", "").strip()
        if len(line) == 0:
            continue
        line = " ".join(jieba.cut(line))
        chinese_list = pattern.findall(line)
        word_list = []
        for word in chinese_list:
            if word not in stop_words:
                word_list.append(word)
        if len(word_list) > 0:
            raw_word_list.extend(word_list)
            line = " ".join(word_list)
            f_writer.write(line + "\n")
            f_writer.flush()
f_writer.close()
vocabulary_set = set(raw_word_list)
print("vocabulary count:", len(vocabulary_set))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\proffl\AppData\Local\Temp\jieba.cache
Loading model cost 1.102 seconds.
Prefix dict has been built successfully.


vocabulary count: 41043


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [ ]:
import collections

vocabulary_size = len(vocabulary_set)
words = raw_word_list
count = [['UNK', '-1']]
count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
print("count", len(count))
count

In [20]:
dictionary = dict()
for word, _ in count:
    dictionary[word] = len(dictionary)
data = list()
unk_count = 0
for word in words:
    if word in dictionary:
        index = dictionary[word]
    else:
        index = 0
        unk_count = unk_count + 1
    data.append(index)
count[0][1] = unk_count
reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
del words
print(reverse_dictionary[1000])
print(data[:200])

江南
[1983, 13891, 20228, 20229, 20230, 7412, 4195, 13892, 20231, 20232, 20233, 13893, 13894, 478, 1810, 10701, 5699, 20234, 5700, 20235, 427, 2433, 10701, 906, 20236, 7413, 20237, 20238, 20239, 3332, 2781, 8739, 251, 2954, 20240, 8740, 7414, 2093, 7415, 10702, 4615, 2434, 1984, 13895, 2094, 3861, 10703, 7416, 13896, 13897, 598, 10703, 8741, 20241, 10703, 509, 1260, 1030, 4615, 2434, 20242, 20243, 132, 20244, 2434, 367, 203, 510, 20245, 13898, 13899, 13900, 4196, 13901, 7417, 839, 5701, 2186, 2435, 20246, 7418, 20247, 10704, 20248, 367, 203, 510, 20249, 2187, 20250, 20251, 105, 6485, 20252, 20253, 5702, 20254, 1622, 10705, 8742, 100, 689, 13902, 13903, 6486, 485, 20255, 2619, 74, 7419, 1559, 587, 1622, 100, 330, 48, 10706, 839, 36, 203, 10707, 4616, 3333, 1811, 20256, 839, 200, 2782, 690, 213, 2783, 20257, 20258, 3334, 4617, 20259, 5091, 551, 3135, 8743, 324, 132, 3335, 10708, 1984, 13904, 1119, 2784, 2785, 4618, 13905, 3565, 2436, 2436, 2188, 2955, 434, 1887, 10709, 20260, 10710, 3566, 

In [24]:
import numpy as np
import random

data_index = 0


def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    batch = np.ndarray(shape=batch_size, dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1

    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels


batch, labels = generate_batch(batch_size=128, num_skips=4, skip_window=2)

# skip-gram model
batch_size = 128
embedding_size = 300
skip_window = 2
num_skips = 4
valid_window = 100
num_sample = 64
learning_rate = 0.01
valid_word = ['令狐冲', '左冷禅', '林平之', '岳不群', '桃根仙']
valid_example = [dictionary[li] for li in valid_word]
data_index = 0


def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    # 得到窗口长度( 当前单词左边和右边 + 当前单词)
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1

    #回溯一点，以避免在批处理结束时跳过单词
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [ ]:
import tensorflow as tf

with tf.device('/cpu:0'):
    # 创建嵌入变量（每一行代表一个词嵌入向量） embedding vector).
    embedding = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
    # 构造NCE损失的变量
    nce_weights = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))


def get_embedding(x):
    with tf.device('/cpu:0'):
        # 对于X中的每一个样本查找对应的嵌入向量
        x_embed = tf.nn.embedding_lookup(embedding, x)
        return x_embed


def nce_loss(x_embed, y):
    with tf.device('/cpu:0'):
        # 计算批处理的平均NCE损失
        y = tf.cast(y, tf.int64)
        loss = tf.reduce_mean(
            tf.nn.nce_loss(weights=nce_weights,
                           biases=nce_biases,
                           labels=y,
                           inputs=x_embed,
                           num_sampled=num_sample,
                           num_classes=vocabulary_size))
        return loss


# 评估
def evaluate(x_embed):
    with tf.device('/cpu:0'):
        # 计算输入数据嵌入与每个嵌入向量之间的余弦相似度
        x_embed = tf.cast(x_embed, tf.float32)
        x_embed_norm = x_embed / tf.sqrt(tf.reduce_sum(tf.square(x_embed)))
        embedding_norm = embedding / tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True), tf.float32)
        cosine_sim_op = tf.matmul(x_embed_norm, embedding_norm, transpose_b=True)
        return cosine_sim_op


# 定义优化器
optimizer = tf.optimizers.SGD(learning_rate)


# 优化过程
def run_optimization(x, y):
    with tf.device('/cpu:0'):
        # 将计算封装在GradientTape中以实现自动微分
        with tf.GradientTape() as g:
            emb = get_embedding(x)
            loss = nce_loss(emb, y)

        # 计算梯度
        gradients = g.gradient(loss, [embedding, nce_weights, nce_biases])

        # 按gradients更新 W 和 b
        optimizer.apply_gradients(zip(gradients, [embedding, nce_weights, nce_biases]))


# 用于测试的单词
x_test = np.array(valid_example)
num_steps = 2000000
avg_loss = 0
# 针对给定步骤数进行训练
for step in range(num_steps):
    batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
    run_optimization(batch_inputs, batch_labels)
    loss = nce_loss(get_embedding(batch_inputs), batch_labels)
    avg_loss = avg_loss + loss

    if step % 5000 == 0:
        if step > 0:
            avg_loss = avg_loss / 5000
        loss = nce_loss(get_embedding(batch_inputs), batch_labels)
        print("step: %i, loss: %f" % (step, loss))
        # print("平均损失在", num_steps, "中为：", avg_loss)

    # 计算验证集合的相似度
    if step % 10000 == 0:
        sim = evaluate(get_embedding(x_test)).numpy()
        for i in range(len(valid_word)):
            val_word = reverse_dictionary[valid_example[i]]
            top_k = 10
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            sim_str = "与" + val_word + "最近的前10词是"
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                sim_str = "%s %s," % (sim_str, close_word)
            print(sim_str)

step: 15000, loss: 449.917725
step: 20000, loss: 382.382935
与令狐冲最近的前10词是 岳不群, 剑法, 师父, 弟子, 岳灵珊, 仪琳, 恒山, 林平之, 盈盈, 田伯光,
与左冷禅最近的前10词是 罢, 半碗, 余丈, 再也, 受, 令狐冲, 围杀, 师公, 叫惯, 出其不意,
与林平之最近的前10词是 令狐冲, 甚烦, 一枝, 绝未, 岳不群, 岳灵珊, 用个, 坚硬, 走走, 向问天,
与岳不群最近的前10词是 令狐冲, 弟子, 剑法, 师父, 岳灵珊, 余沧海, 师哥, 盈盈, 长剑, 仪琳,
与桃根仙最近的前10词是 方形, 着重点, 糊糊, 严惩不贷, 废去, 入门, 谢上, 之纯, 往上, 左掌猛,
step: 25000, loss: 321.458496
step: 30000, loss: 275.788727
与令狐冲最近的前10词是 岳不群, 剑法, 师父, 弟子, 林平之, 盈盈, 岳灵珊, 恒山, 长剑, 仪琳,
与左冷禅最近的前10词是 令狐冲, 岳不群, 恒山, 婆婆, 罢, 教主, 瞧, 师父, 死, 受,
与林平之最近的前10词是 令狐冲, 岳不群, 弟子, 剑法, 师父, 岳灵珊, 盈盈, 恒山, 武功, 师妹,
与岳不群最近的前10词是 令狐冲, 剑法, 弟子, 师父, 盈盈, 岳灵珊, 林平之, 恒山, 长剑, 仪琳,
与桃根仙最近的前10词是 方形, 着重点, 糊糊, 严惩不贷, 废去, 早已, 谢上, 入门, 之纯, 绝大,
step: 35000, loss: 322.641663
step: 40000, loss: 300.589600
与令狐冲最近的前10词是 岳不群, 师父, 弟子, 剑法, 林平之, 岳灵珊, 盈盈, 恒山, 长剑, 田伯光,
与左冷禅最近的前10词是 令狐冲, 岳不群, 恒山, 师父, 弟子, 瞧, 剑法, 婆婆, 仪琳, 教主,
与林平之最近的前10词是 令狐冲, 岳不群, 师父, 弟子, 剑法, 岳灵珊, 盈盈, 恒山, 田伯光, 长剑,
与岳不群最近的前10词是 令狐冲, 弟子, 师父, 剑法, 盈盈, 林平之, 岳灵珊, 恒山, 长剑, 田伯光,
与桃根仙最近的前10词是 方形, 着重点, 糊糊, 早已, 令狐冲, 左冷禅,